In [1]:
import os
import pandas as pd
import plotly.express as px

from plots.simulation.GE3_plots import sankeyplot, emission_data_dict_to_df
from ggmodel_dev.models.landuse import BE2_scenario, GE3_scenario
from ggmodel_dev.models.water import EW_scenario
from ggmodel_dev.models.transport import VEHC_scenario
from ggmodel_dev.projection import run_projection



def get_data_dict_from_folder_parquet(folder_name):
    files = os.listdir(folder_name)
    data_dict = {file.split('.')[0]: pd.read_parquet(
        f'{folder_name}/{file}') for file in files}
    data_dict = {name: df[name] for name, df in data_dict.items()}
    return data_dict


def get_data_dict_from_folder(folder_name):
    files = os.listdir(folder_name)
    data_dict = {file.split('.')[0]: pd.read_csv(
        f'{folder_name}/{file}') for file in files}

    data_dict = {name: df.set_index([col for col in df.columns if col != name]).squeeze(
    ) for name, df in data_dict.items()}

    return data_dict

In [2]:
scenario_data_dictionnary = {
    'EW_models': get_data_dict_from_folder('data/sim/EW'),
    'BE2_model': get_data_dict_from_folder('data/sim/BE2'),
    'GE3_model': get_data_dict_from_folder_parquet('data/sim/GE3'),
    'VEHC_model': get_data_dict_from_folder('data/sim/VEHC'),
}


In [3]:
from ggmodel_dev.projection import *
from ggmodel_dev.models.landuse.GE3 import model_dictionnary

projection_dict = {
    'Pop': lambda x: x,
    'OEi': lambda x: x,
    'EF_EEi': lambda x: x,
    'TAi': lambda x: x,
    'IN_F': lambda x: x,
    'MYi': lambda x: x,
    'EF_ASi': lambda x: x,
    'EF_Ti': lambda x: x,
    'EF_CH4Ti': lambda x: x,
    'EF_Li': lambda x: x,
    'EF_F': lambda x: x,        
}


def run_scenario(data_dict,
                 MM_Ti=1/3,
                 MM_ASi=1/3,
                 MM_LPi=1/3,
                 TAi_pct_Asses=1,
                 TAi_pct_Cattle_dairy=1,
                 TAi_pct_Cattle_non_dairy=1,
                 TAi_pct_Chickens_broilers=1,
                 TAi_pct_Chickens_layers=1,
                 TAi_pct_Ducks=1,
                 TAi_pct_Goats=1,
                 TAi_pct_Horses=1,
                 TAi_pct_Mules=1,
                 TAi_pct_Sheep=1,
                 TAi_pct_Swine_breeding=1,
                 TAi_pct_Swine_market=1,
                 TAi_pct_Turkeys=1
                 ):
    
    data_dict = data_dict.copy()
    
    
    data_dict['MM_Ti'] = MM_Ti
    data_dict['MM_ASi'] = MM_ASi
    data_dict['MM_LPi'] = MM_LPi
    
    TAi_pct_target = pd.Series(index=['Asses', 'Cattle, dairy','Cattle, non-dairy',
                                  'Chickens, broilers', 'Chickens, layers', 'Ducks', 'Goats',
                                  'Horses', 'Mules', 'Sheep', 'Swine, breeding', 'Swine, market',
                                  'Turkeys'],
                           data=[TAi_pct_Asses, TAi_pct_Cattle_dairy,
                                 TAi_pct_Cattle_non_dairy, TAi_pct_Chickens_broilers,
                                 TAi_pct_Chickens_layers, TAi_pct_Ducks,TAi_pct_Goats,
                                 TAi_pct_Horses, TAi_pct_Mules ,TAi_pct_Sheep, TAi_pct_Swine_breeding,
                                 TAi_pct_Swine_market, TAi_pct_Turkeys
                               
                           ], name='TAi_pct_target')
    
    TAi_pct_target.index.name = 'Item'
    
    tmp = pd.merge(data_dict['TAi'].reset_index(), TAi_pct_target.reset_index(),on='Item')
    tmp['TAi'] = tmp['TAi'] * tmp['TAi_pct_target']

    data_dict['TAi'] = tmp.set_index(['ISO', 'Year', 'Item'])['TAi']
    
    results = model_dictionnary['GE3_model'].run(data_dict)
    
    return results

In [99]:
def emission_data_dict_to_df(data_dict):

    data_dict = data_dict.copy()

    data = pd.concat([v.to_frame(name='Value').assign(Variable=k)
                      for k, v in data_dict.items()], axis=0).reset_index().dropna()

    data = pd.concat([data, data.groupby('Variable').sum().reset_index().rename(
        columns={"Variable": 'Item'}).assign(Variable='GE3_partial')])

    return data

In [123]:
def run_all_scenarios_GE3(data_dict, ISO, args_dict_1, args_dict_2):
    scenarios_results = {}
    data_dict = {k: v.loc[ISO, 2018, :] for k, v in data_dict.items()}

    scenarios_results['BAU'] = run_scenario(
        data_dict=data_dict, MM_Ti=data_dict['MM_Ti'], MM_ASi=data_dict['MM_ASi'])
    scenarios_results['scenario_one'] = run_scenario(
        data_dict=data_dict, **args_dict_1)
    scenarios_results['scenario_two'] = run_scenario(
        data_dict=data_dict, **args_dict_2)
    
    displayed_variables = ['TEE_CO2eq', 'TMA_CO2eq', 'TMT_CO2eq', 'TMP_CO2eq']

    df_0 = emission_data_dict_to_df({k: v for k, v in scenarios_results['BAU'].items() if k in displayed_variables})
    df_1 = emission_data_dict_to_df({k: v for k, v in scenarios_results['scenario_one'].items() if k in displayed_variables})
    df_2 = emission_data_dict_to_df({k: v for k, v in scenarios_results['scenario_two'].items() if k in displayed_variables})

    df = pd.concat([df_1.assign(scenario='BAU'), df_1.assign(scenario='scenario 1'), df_2.assign(scenario='scenario 2')], axis=0)
    
    fig_1 = px.treemap(df.query('Variable != "GE3_partial"'), path=['Item','name', 'scenario'], values='Value')

    return fig_1, {}, {}


In [124]:
data_dict = scenario_data_dictionnary['GE3_model']

args_dict_1 = dict(MM_Ti=1/3, MM_ASi=1/3, MM_LPi=1/3)
args_dict_2 = dict(MM_Ti=1/3, MM_ASi=1/3, MM_LPi=1/3, TAi_pct_Cattle_non_dairy=1/2)

df = run_all_scenarios_GE3(data_dict, 'FRA', args_dict_1, args_dict_2)
 

#sankeyplot(df, 'Item', 'Variable')

/home/simon/anaconda3/lib/python3.7/site-packages/pandas/core/indexes/multi.py:3585: RuntimeWarning:

The values in the array are unorderable. Pass `sort=False` to suppress this warning.



In [125]:
from ggmodel_dev.models.landuse.GE3 import model_dictionnary
df = df.merge(model_dictionnary['GE3_model'].summary_df[['name', 'unit']], left_on='Variable', right_index=True, how='left')

df.loc[df.Variable == 'GE3_partial', 'unit'] = 'gigagrams (CO2eq)'
df.loc[df.Variable == 'GE3_partial', 'name'] = 'Non-CO2 agricultural emissions'

# df = df.merge(model_dictionnary['GE3_model'].summary_df[['name']], left_on='Item', right_index=True, how='left', suffixes=('_target', '_source'))
# df['name'] = df['name_target']
# df.loc[df.name_source.isna(), 'name'] = df.loc[df.name_source.isna(), 'Item']

In [126]:
df

,ISO,Year,Item,Value,Variable,scenario,name,unit
1,FRA,2018,Asses,0.871797,TMA_CO2eq,BAU,Emissions (CO2eq) (Manure applied),gigagrams (CO2eq)
3,FRA,2018,"Cattle, dairy",866.338757,TMA_CO2eq,BAU,Emissions (CO2eq) (Manure applied),gigagrams (CO2eq)
4,FRA,2018,"Cattle, non-dairy",1759.820777,TMA_CO2eq,BAU,Emissions (CO2eq) (Manure applied),gigagrams (CO2eq)
6,FRA,2018,"Chickens, broilers",89.383033,TMA_CO2eq,BAU,Emissions (CO2eq) (Manure applied),gigagrams (CO2eq)
7,FRA,2018,"Chickens, layers",86.495942,TMA_CO2eq,BAU,Emissions (CO2eq) (Manure applied),gigagrams (CO2eq)
...,...,...,...,...,...,...,...,...
74,FRA,2018,"Swine, market",377.755906,TEE_CO2eq,scenario 2,Emissions (CO2eq) (Enteric),gigagrams (CO2eq)
0,NaN,18162,TEE_CO2eq,19594.276243,GE3_partial,scenario 2,Non-CO2 agricultural emissions,gigagrams (CO2eq)
1,NaN,26234,TMA_CO2eq,2733.116450,GE3_partial,scenario 2,Non-CO2 agricultural emissions,gigagrams (CO2eq)
2,NaN,22198,TMP_CO2eq,3811.784884,GE3_partial,scenario 2,Non-CO2 agricultural emissions,gigagrams (CO2eq)


In [127]:
import plotly.express as px
fig = px.treemap(df.query('Variable != "GE3_partial"'), path=['Item','name', 'scenario'], values='Value')
fig

In [128]:
from sklearn.preprocessing import LabelEncoder
import plotly.graph_objects as go
import pandas as pd


def emission_data_dict_to_df(data_dict):

    data_dict = data_dict.copy()

    data = pd.concat([v.to_frame(name='Value').assign(Variable=k)
                      for k, v in data_dict.items()], axis=0).reset_index().dropna()

    data = pd.concat([data, data.groupby('Variable').sum().reset_index().rename(
        columns={"Variable": 'Item'}).assign(Variable='Non-CO2 agricultural emissions')])

    return data


def encode_source_target(df, source='Item', target='Variable'):

    le = LabelEncoder()

    encoded = le.fit_transform(
        df[[source, target]].values.flatten()).reshape(-1, 2)

    return encoded, le.classes_


def sankeyplot(df, source, target, valueformat='.00f', valuesuffix=' Gg(CO2eq)'):

    data = df.copy()

    encoded_s_t, classe_names = encode_source_target(data, source, target)

    data[['Source', 'Target']] = encoded_s_t
    color_dict = pd.DataFrame({'scenario': ['BAU', 'scenario 1' ,'scenario 2'],
                               'color': ['grey', '#D8A488', '#86BBD8']})
    
    data = data.merge(color_dict, on='scenario')
    
    fig = go.Figure(data=[go.Sankey(
        valueformat=valueformat,
        valuesuffix=valuesuffix,
        node=dict(
            pad=15,
            thickness=20,
            line=dict(color="black", width=0.5),
            label=classe_names,
            color='lightgrey'
        ),
        link=dict(
            target=data['Target'],
            source=data['Source'],
            value=data['Value'],
            color=data['color']
        ))])

    return fig

In [129]:
df

,ISO,Year,Item,Value,Variable,scenario,name,unit
1,FRA,2018,Asses,0.871797,TMA_CO2eq,BAU,Emissions (CO2eq) (Manure applied),gigagrams (CO2eq)
3,FRA,2018,"Cattle, dairy",866.338757,TMA_CO2eq,BAU,Emissions (CO2eq) (Manure applied),gigagrams (CO2eq)
4,FRA,2018,"Cattle, non-dairy",1759.820777,TMA_CO2eq,BAU,Emissions (CO2eq) (Manure applied),gigagrams (CO2eq)
6,FRA,2018,"Chickens, broilers",89.383033,TMA_CO2eq,BAU,Emissions (CO2eq) (Manure applied),gigagrams (CO2eq)
7,FRA,2018,"Chickens, layers",86.495942,TMA_CO2eq,BAU,Emissions (CO2eq) (Manure applied),gigagrams (CO2eq)
...,...,...,...,...,...,...,...,...
74,FRA,2018,"Swine, market",377.755906,TEE_CO2eq,scenario 2,Emissions (CO2eq) (Enteric),gigagrams (CO2eq)
0,NaN,18162,TEE_CO2eq,19594.276243,GE3_partial,scenario 2,Non-CO2 agricultural emissions,gigagrams (CO2eq)
1,NaN,26234,TMA_CO2eq,2733.116450,GE3_partial,scenario 2,Non-CO2 agricultural emissions,gigagrams (CO2eq)
2,NaN,22198,TMP_CO2eq,3811.784884,GE3_partial,scenario 2,Non-CO2 agricultural emissions,gigagrams (CO2eq)


In [130]:
sankeyplot(df, 'Item', 'Variable')